In [1]:
# %pip install albumentations

In [2]:
%cd /notebooks/Pix2Pix

/notebooks/Pix2Pix


In [3]:
%ls

README.md         dataset_maps.py               pix2pix-April_15.ipynb
__pycache__/      disc.pth.tar                  results/
config.py         disc_SEN12MS_april30.pth.tar  saved_images/
data/             discriminator_model.py        saved_images.zip
data_anime/       evaluation/                   test_drone_images/
data_maps/        gen.pth.tar                   train.py
dataset.py        gen_SEN12MS_april30.pth.tar   utils.py
dataset_anime.py  generator_model.py


In [6]:
!python3 /notebooks/Pix2Pix/train.py

100%|████████████| 9033/9033 [15:48<00:00,  9.53it/s, D_fake=0.148, D_real=0.93]
=> Saving checkpoint
=> Saving checkpoint
100%|███████████| 9033/9033 [16:13<00:00,  9.28it/s, D_fake=0.324, D_real=0.591]
=> Saving checkpoint
=> Saving checkpoint
100%|███████████| 9033/9033 [16:37<00:00,  9.06it/s, D_fake=0.115, D_real=0.857]
=> Saving checkpoint
=> Saving checkpoint
100%|███████████| 9033/9033 [15:56<00:00,  9.44it/s, D_fake=0.255, D_real=0.511]
=> Saving checkpoint
=> Saving checkpoint
 17%|█▉         | 1569/9033 [02:44<13:02,  9.54it/s, D_fake=0.166, D_real=0.932]
Traceback (most recent call last):
  File "/notebooks/pix2pix/Pix2Pix/train.py", line 99, in <module>
    main()
  File "/notebooks/pix2pix/Pix2Pix/train.py", line 87, in main
    train_fn(
  File "/notebooks/pix2pix/Pix2Pix/train.py", line 48, in train_fn
    g_scaler.step(opt_gen)
  File "/usr/local/lib/python3.9/dist-packages/torch/cuda/amp/grad_scaler.py", line 338, in step
    retval = self._maybe_opt_step(optimizer, o

In [8]:
%pip install torchsummary


Note: you may need to restart the kernel to use updated packages.


In [15]:
import os
import torch
import numpy as np
from PIL import Image
from torchsummary import summary
from generator_model import Generator
from torchvision import transforms, utils
import matplotlib.pyplot as plt

In [22]:
# model = torch.load('gen.pth.tar', map_location='cuda')  # Load directly to GPU
model = Generator(in_channels=3, features=64)
model_data = torch.load('gen_SEN12MS_april30.pth.tar', map_location='cuda')  # Load directly to CPU (map_location='cpu') or GPU (map_location='cuda')
# model.eval() *args, **kwargs
print("Keys in loaded model data:", model_data.keys())
# model = torch.nn.Module()
model.load_state_dict(model_data['state_dict'])
model.eval()
print()

Keys in loaded model data: dict_keys(['state_dict', 'optimizer'])



In [17]:
def save_and_display_image(image, title, epoch, folder, filename):
    # Denormalize and prepare the image
    image = image * 0.5 + 0.5  # Rescale from [-1, 1] to [0, 1]
    utils.save_image(image, os.path.join(folder, f"{filename}_{epoch}.png"))

    # Display the image
    plt.imshow(image.permute(1, 2, 0).numpy())
    plt.title(title)
    plt.axis('off')

In [18]:
def predict(img_path, output_folder='./saved_images'):
    # Load the image
    image = Image.open(img_path)
    transform = transforms.Compose([
        # transforms.Resize(3948),
        # transforms.CenterCrop(3948),
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    input_image = transform(image).unsqueeze(0)

    # Directory for saving images
    os.makedirs(output_folder, exist_ok=True)

    # Generate the output
    with torch.no_grad():
        output_image = model(input_image)  # Assume model is globally defined or passed as a parameter
        output_image = output_image * 0.5 + 0.5  # Rescale to [0, 1] if using tanh in the generator

        # Construct a valid output filename
        filename = os.path.basename(img_path)
        filename_without_ext = os.path.splitext(filename)[0]
        output_path = os.path.join(output_folder, f"generated_image_{filename_without_ext}.png")

        utils.save_image(output_image, output_path)

    print(f"Image saved to {output_path}")

In [19]:
'''
This function tests the GAN by inputing test Drone captured images from folder 'test_drone_images'
into the generator and saves those images to './saved_images'
'''

# # Call the function
# for image in drone_images.

for filename in os.listdir('test_drone_images'):
    # Construct the full file path
    img_path = os.path.join('test_drone_images', filename)

    # Check if the file is an image (optional, based on file extension)
    if img_path.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        # Process the image
        predict(img_path)
    else:
        print(f"Skipped non-image file: {filename}")

print("All images have been processed.")

Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-4737.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-109.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-104.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-4743.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-34.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-84.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-59.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-89.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-4738.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-54.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-4740.png
Image saved to ./saved_images/generated_image_Waterville Brown Tail 2022-94.png
Image saved to ./saved_images/